## install libraries

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## library import

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

GLOBAL_SEED = 42

## load dataset

In [ ]:
dataset = load_dataset("persiannlp/parsinlu_query_paraphrasing")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1830
    })
    test: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1916
    })
    validation: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 898
    })
})

In [ ]:
dataset["test"]

Dataset({
    features: ['q1', 'q2', 'category', 'label'],
    num_rows: 1916
})

In [ ]:
train_df = pd.DataFrame(dataset["train"]) # for examples in one-shot and few-shot learning
test_df = pd.DataFrame(dataset["test"]) # for evaluation
test_df

,q1,q2,category,label
0,آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنتر...,چه چیزی روح فرد را می شکند؟,qqp,0
1,چگونه می توانم تماشای فیلم های پورنو را متوقف ...,برای جلوگیری از تماشای کامل پورنو باید چه کاری...,qqp,1
2,چه کسانی امام علی را خدا میدانند؟,چه کسانی می توانند امام زمان را ببینند؟,natural,0
3,آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایال...,شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا...,qqp,1
4,چگونه می توانم سوالی را در این باره بپرسم؟,چگونه می توانم سوال بپرسم؟,qqp,0
...,...,...,...,...
1911,چه چیزهایی روزه زن را باطل میکند اهل سنت؟,چه چیزهایی روزه زن را باطل میکند مکارم شیرازی؟,natural,0
1912,وای چه پسری وای عجب تاج سری؟,دانلود اهنگ پسر وای وای چه پسری وای عجب تاج سری؟,natural,1
1913,از چه زمانی حرکات جنین کم میشود؟,از چه زمانی حرکات جنین حس میشود؟,natural,0
1914,کدام یک از شرکتها دارای رشد شغلی بهتری دارند؟ ...,۴ لوله ورودی و ۲ لوله خروجی ، ورودی آب را پیدا...,qqp,0


## test model on a single example

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "ViraIntelligentDataMining/PersianLLaMA-13B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(model_name)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
# )
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    low_cpu_mem_usage=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # quantization_config=quantization_config, #
    load_in_4bit=True
)

tokenizer_config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00003-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     "universitytehran/PersianMind-v1.0",
#     torch_dtype=torch.bfloat16,
#     low_cpu_mem_usage=True,
#     device_map={"": device},
# )
# # # quantize the model in 8-bit (INT8)
# # model = AutoModelForCausalLM.from_pretrained(
# #     "universitytehran/PersianMind-v1.0",
# #     device_map="auto",
# #     low_cpu_mem_usage=True,
# #     load_in_8bit=True
# # )
# # # quantize the model in 4-bit (NormalFloat4)
# # quantization_config = BitsAndBytesConfig(
# #     load_in_4bit=True,
# #     bnb_4bit_use_double_quant=True,
# #     bnb_4bit_quant_type="nf4",
# # )
# # model = AutoModelForCausalLM.from_pretrained(
# #     "universitytehran/PersianMind-v1.0",
# #     quantization_config=quantization_config,
# #     device_map="auto"
# # )


# tokenizer = AutoTokenizer.from_pretrained(
#     "universitytehran/PersianMind-v1.0",
# )

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/688k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

In [ ]:
!ls "/root/.cache/huggingface/hub/"

models--ViraIntelligentDataMining--PersianLLaMA-13B  version.txt


In [ ]:
!rm -rf "/root/.cache/huggingface/hub/models--ViraIntelligentDataMining--PersianLLaMA-13B"

In [ ]:
!ls "/root/.cache/huggingface/hub/"

version.txt


In [ ]:
prompt = "این متن به فارسی است"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs["input_ids"], max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

این متن به فارسی است و در سال‌های اخیر به زبان‌های دیگر ترجمه شده‌است.  [NEW_LINE] این کتاب در سال‌های اخیر به زبان‌های دیگر ترجمه شده‌است.  [NEW_LINE] ترجمه‌ها.  [NEW_LINE] این کتاب به زبان‌های زیر ترجمه شده‌است:  [NEW_LINE] منابع.  [NEW_LINE] 


In [ ]:
prompt = "در ادامه دو سوال آمده است. " + \
            "آیا این دو سوال مترادف هستند یا خیر؟ " + \
            "اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. " + \
            "اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. " + \
            f"سوال اول: {train_df.iloc[100]['q1']} " + \
            f"سوال دوم: {train_df.iloc[100]['q2']}"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs["input_ids"], max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):])

  [NEW_LINE] در ادامه دو سؤال آمده است. آیا این دو سؤال مترادف هستند یا خیر؟ اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. سؤال اول: آیا به دارو حساسیت داریم؟ سؤال دوم: آیا به گلوتن حساسیت داریم؟  [NEW_LINE] در ادامه دو سؤال آمده است. آیا این دو سؤال مترادف هستند یا خیر؟ اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. سؤال


In [ ]:
train_df.iloc[100]['q1']

'چگونه بفهمیم به داروی بیهوشی حساسیت داریم؟'

In [ ]:
train_df.iloc[100]['q2']

'چگونه بفهمیم به گلوتن حساسیت داریم؟'

In [ ]:
label_mapping = {
    0: "خیر",
    1: "بله"
}

## inference on different scenarios

In [ ]:
def create_prompt(scenario, q1, q2, examples=None):
    if scenario == "Zero-shot":
        return  "در ادامه دو سوال آمده است. " + \
                    "آیا این دو سوال مترادف هستند یا خیر؟ " + \
                    "اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. " + \
                    "اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. " + \
                    f"سوال اول: '{q1}' " + \
                    f"سوال دوم: '{q2}'"
    elif scenario == "One-shot":
        return  "در ادامه دو سوال آمده است. " + \
                    "آیا این دو سوال مترادف هستند یا خیر؟ " + \
                    "اگر مترادف هستند فقط در یک کلمه 'بله' را تولید کنید. " + \
                    "اگر مترادف نیستند فقط در یک کلمه 'خیر' را تولید کنید. " + \
                    "مثال: " + \
                    f"سوال اول: '{examples.iloc[0]['q1']}' " + \
                    f"سوال دوم: '{examples.iloc[0]['q2']}' " + \
                    f"جواب: '{label_mapping[int(examples.iloc[0]['label'])]}' " + \
                    "حال با دیدن این مثال جواب دهید. " + \
                    f"سوال اول: '{q1}' " + \
                    f"سوال دوم: '{q2}'"
    elif scenario == "Five-shot":
        examples_text = ""
        for i, example in examples.iterrows():
            examples_text += f"سوال اول: '{example['q1']}' " + \
                               f"سوال دوم: '{example['q2']}' " + \
                               f"جواب: '{label_mapping[int(example['label'])]}' "
        return  "در ادامه دو سوال آمده است. " + \
                    "آیا این دو سوال مترادف هستند یا خیر؟ " + \
                    "اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. " + \
                    "اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. " + \
                    "مثال ها: " + \
                    f"{examples_text} " + \
                    "حال با دیدن مثال ها جواب دهید. " + \
                    f"سوال اول: '{q1}' " + \
                    f"سوال دوم: '{q2}'"

In [ ]:
def generate_output_from_prompt(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):]

In [ ]:
# test our functions
test_subset_examples = test_df.sample(n=100, random_state=GLOBAL_SEED) # because it takes too much to test all examples we use subset of it
one_shot_subset_examples = train_df.sample(n=1, random_state=GLOBAL_SEED)
five_shot_subset_examples = train_df.sample(n=5, random_state=GLOBAL_SEED)

print(create_prompt("Zero-shot", test_df.iloc[85]['q1'], test_df.iloc[85]['q2'], None))
print()
print(create_prompt("One-shot", test_df.iloc[85]['q1'], test_df.iloc[85]['q2'], one_shot_subset_examples))
print()
print(create_prompt("Five-shot", test_df.iloc[85]['q1'], test_df.iloc[85]['q2'], five_shot_subset_examples))

در ادامه دو سوال آمده است. آیا این دو سوال مترادف هستند یا خیر؟ اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. سوال اول: 'چه دارو گیاهی بخوریم تا چاق شویم؟' سوال دوم: 'چه چیزی ناشتا بخوریم تا چاق شویم؟'

در ادامه دو سوال آمده است. آیا این دو سوال مترادف هستند یا خیر؟ اگر مترادف هستند فقط در یک کلمه 'بله' را تولید کنید. اگر مترادف نیستند فقط در یک کلمه 'خیر' را تولید کنید. مثال: سوال اول: 'آیا می توان انرژی مکانیکی را حفظ کرد؟' سوال دوم: 'چه زمانی و چگونه انرژی مکانیکی حفظ نمی شود؟' جواب: 'بله' حال با دیدن این مثال جواب دهید. سوال اول: 'چه دارو گیاهی بخوریم تا چاق شویم؟' سوال دوم: 'چه چیزی ناشتا بخوریم تا چاق شویم؟'

در ادامه دو سوال آمده است. آیا این دو سوال مترادف هستند یا خیر؟ اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. مثال ها: سوال اول: 'آیا می توان انرژی مکانیکی را حفظ کرد؟' سوال دوم: 'چه زمانی و چگونه انرژی مکانیکی حفظ نمی شود؟' جواب: 'بله

In [ ]:
def find_first_label(labels, text):
    # Initialize a dictionary to store the position of each label
    positions = {label: text.find(label) for label in labels}

    # Filter out labels that are not found (position -1)
    valid_positions = {label: pos for label, pos in positions.items() if pos != -1}

    # If no valid positions are found, return None
    if not valid_positions:
        return None

    # Find the label with the minimum position
    first_label = min(valid_positions, key=valid_positions.get)

    return first_label

In [ ]:
def test_scenario(scenario, test_df, log_df, examples=None):
    correct = 0
    total = len(test_df)
    labels = ["بله", "خیر"]

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"Running {scenario} scenario..."):
        prompt = create_prompt(scenario, row['q1'], row['q2'], examples)
        output = generate_output_from_prompt(prompt)
        label = int(row['label'])
        if label == 1:
            # if output == "بله":
            # if "بله" in output.split(" ")[0]:
            # if "بله" in output:
            if "بله" == find_first_label(labels, output):
                correct += 1
        else:
            # if output == "خیر":
            # if "خیر" in output.split(" ")[0]:
            # if "خیر" in output:
            if "خیر" == find_first_label(labels, output):
                correct += 1
        log_df.loc[len(log_df)] = [row['q1'], row['q2'], output, label_mapping[label], scenario]

    return correct / total

In [25]:
log_df = pd.DataFrame(columns=["q1", "q2", "generated_text", "real_label", "scenario"])

scenarios = ["Zero-shot", "One-shot", "Five-shot"]

test_subset_examples = test_df.sample(n=50, random_state=GLOBAL_SEED) # because it takes too much to test all examples we use subset of it
one_shot_subset_examples = train_df.sample(n=1, random_state=GLOBAL_SEED)
# five_shot_subset_examples = train_df.sample(n=5, random_state=GLOBAL_SEED)
# balance between 0 and 1
grouped = train_df.groupby('label')
five_shot_subset_examples = grouped.apply(lambda x: x.sample(n=2 if x.name == '0' else 3, random_state=GLOBAL_SEED))
five_shot_subset_examples = five_shot_subset_examples.reset_index(drop=True)


for scenario in scenarios:
    if scenario == "Zero-shot":
        zero_shot_result = test_scenario(scenario, test_subset_examples, log_df,examples=None)
    elif scenario == "One-shot":
        one_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=one_shot_subset_examples)
    elif scenario == "Five-shot":
        five_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=five_shot_subset_examples)
    else:
        # exception
        raise ValueError("Invalid scenario")

model_id = "PersianLLaMA-13B-Instruct"
print("Experiment result:")
print(f"Model Name: {model_id}")
print(f"Zero-shot accuracy: {zero_shot_result}")
print(f"One-shot accuracy: {one_shot_result}")
print(f"Five-shot accuracy: {five_shot_result}")

Running Five-shot scenario...: 100%|██████████| 50/50 [15:25<00:00, 18.51s/it]

Experiment result:
Model Name: PersianLLaMA-13B-Instruct
Zero-shot accuracy: 0.64
One-shot accuracy: 0.64
Five-shot accuracy: 0.64


In [26]:
# save log_df
model_name = "PersianLLaMA-13B-Instruct"
log_df.to_csv(f"log_df_{model_name}.csv", index=False, encoding="utf-8-sig")

In [27]:
log_df

,q1,q2,generated_text,real_label,scenario
0,ويتامين ب در چه چيزهايي وجود دارد؟,ويتامين b12 در چه چيزهايي وجود دارد؟,[NEW_LINE] در ادامه دو سؤال آمده است. آیا ای...,خیر,Zero-shot
1,داستان معراج پیامبر در کدام سوره است؟,داستان معراج پیامبر در کدام سوره قرانی امده است؟,[NEW_LINE] در ادامه دو سؤال آمده است. آیا ای...,بله,Zero-shot
2,کارتهای اعتباری در مشاغل کوچک و رستوران های ما...,کارتهای اعتباری در مشاغل کوچک و رستورانهای اسر...,[NEW_LINE] در ادامه دو سؤال آمده است. آیا ای...,خیر,Zero-shot
3,زیرنویس فارسی فیلم دره من چه سرسبز بود؟,دانلود زیرنویس فارسی فیلم دره من چه سرسبز بود؟,[NEW_LINE] در ادامه دو سؤال آمده است. آیا ای...,بله,Zero-shot
4,از کجا بفهمیم باتری لپ تاپ سالم است؟,از کجا بفهمیم باتری لپ تاپ خراب است؟,[NEW_LINE] در ادامه دو سؤال آمده است. آیا ای...,خیر,Zero-shot
...,...,...,...,...,...
145,سود سهام عدالت شامل چه کسانی می شود؟,سود سهام عدالت به چه کسانی پرداخت می شود؟,جواب: 'خیر' [NEW_LINE] (۱) [NEW_LINE] (۲)...,بله,Five-shot
146,از چه زمانی نماز خواندن واجب شد؟,نماز از چه زمانی بر مسلمانان واجب شد؟,جواب: 'خیر' [NEW_LINE] سؤال اول: «از چه زمان...,بله,Five-shot
147,به روزرسانی در مورد باهوبالی - نتیجه گیری؟,برخی از واقعیت های دیوانه کننده درباره فیلم با...,جواب: 'خیر' [NEW_LINE] (۱) [NEW_LINE] (۲)...,خیر,Five-shot
148,روند ویزای H1-B به دلیل ریاست جمهوری ترامپ چگو...,تفاوت ویزای H1 و H1-B در چیست؟,جواب: 'خیر' [NEW_LINE] (۱) [NEW_LINE] (۲)...,خیر,Five-shot


In [28]:
# save the printed result in a csv file
results = {
    "Model Name": [model_id],
    "Zero-shot accuracy": [zero_shot_result],
    "One-shot accuracy": [one_shot_result],
    "Five-shot accuracy": [five_shot_result]
}

df = pd.DataFrame(results)

df.to_csv(f"experiment_results_{model_name}.csv", index=False)

In [29]:
df

,Model Name,Zero-shot accuracy,One-shot accuracy,Five-shot accuracy
0,PersianLLaMA-13B-Instruct,0.64,0.64,0.64
